In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        x = self.pool3(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 256 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Set batch size
batch_size = 2048

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Initialize the CNN model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
epochs = 300
init_start_time = time.time()

for epoch in range(epochs):
    start_time = time.time()
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch: {epoch+1}, Loss: {running_loss/len(trainloader)}, Training Time: {training_time:.2f} seconds')

    total_end_time = time.time()
    total_train_time = total_end_time - init_start_time
    print(f'Total Training Time: {total_train_time:.2f} seconds')

    # Validation accuracy
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {100*accuracy:.2f}%')

    model.train()

print(f'Final Test Accuracy: {100*accuracy:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch: 1, Loss: 2.303634719848633, Training Time: 17.66 seconds
Total Training Time: 17.66 seconds
Test Accuracy: 9.96%
Total Training Time: 51.16 seconds
Test Accuracy: 10.12%
Total Training Time: 70.05 seconds
Test Accuracy: 10.46%
Total Training Time: 88.69 seconds
Test Accuracy: 10.72%
Total Training Time: 124.76 seconds
Test Accuracy: 10.96%
Total Training Time: 158.30 seconds
Test Accuracy: 11.28%
Total Training Time: 193.27 seconds
Test Accuracy: 11.51%
Total Training Time: 231.32 seconds
Test Accuracy: 11.69%
Total Training Time: 327.84 seconds
Test Accuracy: 11.73%
Total Training Time: 458.17 seconds
Test Accuracy: 11.88%
Epoch: 11, Loss: 2.301841993331909, Training Time: 23.01 seconds
Total Training Time: 490.11 seconds
Test Accuracy: 12.02%
Total Training Time: 517.09 seconds
Test Accuracy: 12.16%
Total Training Time: 541.80 seconds
Test Accuracy: 12.20%
Total Tr